### Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Import file

In [11]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(html)[0]
df.to_csv('beautifulsoup_pandas.csv', header = 0, index = False)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove every row with 'Not assigned' under Borough

In [12]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine rows with same postcodes... same postcodes can have multiple neighborhoods

In [23]:
df = df.groupby(['Postcode', 'Borough'], sort = False).agg(','.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Neighborhood is same as Borough if Neighbourhood value is 'Not assigned'

In [21]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Geographical Coordinates from CSV File

In [24]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
dfC = pd.read_csv('Toronto_long_lat_data.csv')
dfC.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Import Libraries

In [25]:
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors
 
from sklearn.cluster import KMeans
import folium 

Merge Tables

In [37]:
dfC.columns = ['Postcode', 'Latitude', 'Longitude']

dfC = pd.merge(df, dfC[['Postcode', 'Latitude', 'Longitude']], on = 'Postcode')

dfC.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [38]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = "Toronto")
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print('The geographical coordinates of Toronto are Latitude {} and Longitude {}'.format(lat, long))

The geographical coordinates of Toronto are Latitude 43.653963 and Longitude -79.387207


In [40]:
map_toronto = folium.Map(location = [lat, long], zoom_start = 11)

for lat, lng, borough, Neighbourhood in zip(dfC['Latitude'], dfC['Longitude'], dfC['Borough'], dfC['Neighbourhood']):
    label = "{}, {}".format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_toronto)
    
map_toronto

K-Means

In [43]:
kclus = 5

dfC_cluster = dfC.drop('Neighbourhood', 1)
dfC_cluster = dfC_cluster.drop('Borough', 1)
dfC_cluster = dfC_cluster.drop('Postcode', 1)

kmeans = KMeans(n_clusters = kclus, random_state = 0).fit(dfC_cluster)

kmeans.labels_[1:10]

array([4, 2, 3, 2, 1, 0, 4, 4, 2], dtype=int32)

In [44]:
cluster_labels = pd.DataFrame(kmeans.labels_)[0]
cluster_labels.head()

0    4
1    4
2    2
3    3
4    2
Name: 0, dtype: int32

In [45]:
dfC['Cluster'] = cluster_labels

In [46]:
dfC = pd.DataFrame(dfC)
dfC.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,3
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2


Create Map with clusters

In [49]:
map_clusters = folium.Map(location = [lat, long], zoom_start = 10)

x = np.arange(kclus)
ys = [i + x + (i*x) ** 2 for i in range(kclus)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

marker_color = []
for lat, lng, poi, cluster in zip(dfC["Latitude"], dfC['Longitude'], dfC['Neighbourhood'], dfC['Cluster']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters